In [40]:
import pandas as pd
import json

from typing import Any, Literal
from pathlib import Path

In [41]:
def parse_results(file_name: str) -> pd.DataFrame:
    records = []
    with open(f"./results/{file_name}", 'r') as f:
        buffer = []
        depth = 0

        for raw in f:
            line = raw.strip()
            if not line:
                continue  # skip blank lines

            # Track nesting of braces
            depth += line.count('{')
            depth -= line.count('}')

            buffer.append(raw)
            # If we’ve closed all opened braces, process one full JSON object
            if depth == 0 and buffer:
                chunk = ''.join(buffer)
                obj = json.loads(chunk)
                # same flattening logic as before
                for uid, details in obj.items():
                    params = details.pop('params')
                    flat = {'id': uid, **details}
                    for p, v in params.items():
                        if isinstance(v, list) and len(v) == 2:
                            flat[f'{p}_x'] = v[0]
                            flat[f'{p}_y'] = v[1]
                        else:
                            flat[p] = v
                    records.append(flat)
                buffer = []
    return pd.DataFrame(records)

In [42]:
def pick_best_result(df_results: pd.DataFrame) -> pd.DataFrame:
    # Find the index of the row with the lowest score
    best_index = df_results['score'].idxmin()
    # Return the row as a DataFrame
    return df_results.loc[[best_index]].reset_index(drop=True)

In [43]:
def _get_config(file_name: str) -> dict[str, Any]:
    stem = Path(file_name).stem            
    config_name = stem.removeprefix("result_")      
    
    config_path = Path("configurations") / f"{config_name}.json"
    
    # 3) Open & load the JSON
    with config_path.open("r", encoding="utf-8") as f:
        config = json.load(f)
    return config

In [44]:
def parse_config(file_name: str) -> pd.DataFrame:
    config = _get_config(file_name)
    solver: str = config["solverName"]
    algorithm: str = config["reconstruction_algo"]
    options = {option: [value] for option, value in config["optimizer"].items() if option not in ("objective", "nested", "weights")}
    weigths = {f"weight_{i}": [w] for i, w in enumerate(config["optimizer"]["weights"])}
    return pd.DataFrame({"solver": [solver],
                        "algorithm": [algorithm],
                        "hyperparams": [{"solver": solver,**{k: v[0] for k, v in options.items()}}],
                        **options,
                        **weigths}).reset_index(drop=True)

In [45]:
def load_experiment(file_name: str) -> pd.DataFrame:
    df_results = parse_results(file_name)
    best_params = pick_best_result(df_results)
    df_config = parse_config(file_name)
    
    # Combine the best parameters with the configuration
    combined_df = pd.concat([df_config, best_params], axis=1)
    
    # Add the file name as a new column
    combined_df['file_name'] = file_name
    
    return combined_df

In [46]:
def load_experiments_with(algorithm: Literal["TF", "ST", "GDFS"], weights: tuple[float, float, float, float]) -> pd.DataFrame:
    results_dir = Path("./results")
    all_files = [f for f in results_dir.glob("*.jsonl") if f.is_file()]
    
    all_experiments: list[Any] = []
    for file in all_files:
        config = _get_config(file.name)
        if config["reconstruction_algo"] == algorithm and tuple(config["optimizer"]["weights"]) == weights:
            df_experiment = load_experiment(file.name)
            all_experiments.append(df_experiment)

    return pd.concat(all_experiments, ignore_index=True)

In [47]:
def pick_top_k_config(df_experiments: pd.DataFrame, k: int) -> pd.DataFrame:
    # Sort by score (ascending) and take the top k rows
    sorted_df = df_experiments.sort_values('score', ascending=True)
    # Return the top k rows as a DataFrame
    return sorted_df.head(k).reset_index(drop=True)

In [48]:
experiments: list[dict[str, Any]] = [
    {"algorithm": "ST", "weights": (0.1, 0.3, 0.5, -10.0)},
    {"algorithm": "ST", "weights": (0.01, 0.5, 0.5, -7.0)},
    {"algorithm": "ST", "weights": (0.4, 0.2, 0.5, -7.0)},
    {"algorithm": "ST", "weights": (0.4, 0.2, 0.3, -5.0)},
]
success_counts = {}
for experiment in experiments:
    df_experiments = load_experiments_with(experiment["algorithm"], experiment["weights"])
    top_1 = pick_top_k_config(df_experiments, 1)
    top_1 = best_config.dropna(axis=1)
    hkey = json.dumps(top_1.iloc[0]['hyperparams'])
    success_counts[hkey] = success_counts.get(hkey, 0) + 1
success_counts
    

{'{"solver": "GridSearch", "resolution": 10}': 4}